In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 원본 경로
DRIVE_DIR = "/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet"
ZIP_PATH  = f"{DRIVE_DIR}/PP-PicoDet.zip"

# 로컬 대상 경로
LOCAL_ZIP  = "/content/PP-PicoDet.zip"

# ZIP 복사 (진행률 표시)
!rsync -ah --info=progress2 "{ZIP_PATH}" "{LOCAL_ZIP}"

        628.51M 100%   88.49MB/s    0:00:06 (xfr#1, to-chk=0/1)


In [ ]:
!python -m pip install paddlepaddle-gpu==2.6.0.post117 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html > /dev/null

In [ ]:
import os
import sys
import zipfile

In [ ]:
if not os.path.exists('/content/PaddleDetection'):
    print(">>> Cloning PaddleDetection Repository...")
    !git clone https://github.com/PaddlePaddle/PaddleDetection.git

    # 의존성 라이브러리 설치
    print(">>> Installing Dependencies...")
    %cd /content/PaddleDetection
    !pip install -r requirements.txt > /dev/null
    !python setup.py install > /dev/null
    print("✅ PaddleDetection installed.")
else:
    print("✅ PaddleDetection already exists. Skipping clone.")
    %cd /content/PaddleDetection

>>> Cloning PaddleDetection Repository...
Cloning into 'PaddleDetection'...
remote: Enumerating objects: 258356, done.
remote: Counting objects: 100% (784/784), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 258356 (delta 634), reused 397 (delta 397), pack-reused 257572 (from 3)
Receiving objects: 100% (258356/258356), 422.20 MiB | 52.83 MiB/s, done.
Resolving deltas: 100% (210520/210520), done.
>>> Installing Dependencies...
/content/PaddleDetection
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 

In [ ]:
LOCAL_ZIP = "/content/PP-PicoDet.zip"
EXTRACT_DIR = "/content/PP-PicoDet_dataset"
os.makedirs(EXTRACT_DIR, exist_ok=True)

with zipfile.ZipFile(LOCAL_ZIP, "r") as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

print("✅ 압축 해제 완료:", EXTRACT_DIR)


✅ 압축 해제 완료: /content/PP-PicoDet_dataset


In [ ]:
import paddle
import ppdet
import cv2
import matplotlib.pyplot as plt
import json
import numpy as np
import glob
from paddle.vision.transforms import Resize

[02/02 23:48:57] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.


In [ ]:
print(f"📌 Paddle Version: {paddle.__version__}")
print(f"📌 CUDA Available: {paddle.is_compiled_with_cuda()}")

📌 Paddle Version: 2.6.0
📌 CUDA Available: True


In [ ]:
# 1. PaddleDetection 폴더로 이동
%cd /content/PaddleDetection

/content/PaddleDetection


In [ ]:
# 파일 경로
config_path = '/content/PP-PicoDet_dataset/custom_picodet.yml'

# ★ 최신 PaddleDetection 호환용 클린 설정 (불필요한 파라미터 전부 제거)
config_content = """
# =========================================================
# [PicoDet-S Final Fix for GT Batch Error]
# =========================================================
use_gpu: true
epoch: 100
log_iter: 20
save_dir: output
snapshot_epoch: 5
metric: COCO

num_classes: 53

architecture: PicoDet
pretrain_weights: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model_m_416/best_model/best_416_m_before.pdparams

PicoDet:
  backbone: LCNet
  neck: CSPPAN
  head: PicoHead

LCNet:
  scale: 1.0
  feature_maps: [3, 4, 5]

CSPPAN:
  out_channels: 96

PicoHead:
  conv_feat:
    name: PicoFeat
    feat_in: 96
    feat_out: 96
    num_convs: 2
    num_fpn_stride: 4
    norm_type: bn
    share_cls_reg: True
  fpn_stride: [8, 16, 32]
  feat_in_chan: 96
  prior_prob: 0.01
  reg_max: 7
  nms:
    name: MultiClassNMS
    nms_top_k: 1000
    keep_top_k: 100
    score_threshold: 0.025
    nms_threshold: 0.6
  loss_class:
    name: VarifocalLoss
    use_sigmoid: True
    iou_weighted: True
    loss_weight: 1.0
  loss_dfl:
    name: DistributionFocalLoss
    loss_weight: 0.25
  loss_bbox:
    name: GIoULoss
    loss_weight: 2.0

# ---------------------------------------------------------
# 데이터 경로
# ---------------------------------------------------------
TrainDataset:
  !COCODataSet
    dataset_dir: /content/PP-PicoDet_dataset
    image_dir: images_640/train
    anno_path: annotations/instances_train.json
    data_fields: ['image', 'gt_bbox', 'gt_class', 'is_crowd']

EvalDataset:
  !COCODataSet
    dataset_dir: /content/PP-PicoDet_dataset
    image_dir: images_640/val
    anno_path: annotations/instances_val.json

TestDataset:
  !ImageFolder
    dataset_dir: /content/PP-PicoDet_dataset
    anno_path: annotations/instances_test.json

# ---------------------------------------------------------
# 데이터 로더 (PadGT 추가됨)
# ---------------------------------------------------------
worker_num: 4
eval_height: &eval_height 640
eval_width: &eval_width 640
eval_size: &eval_size [*eval_height, *eval_width]

TrainReader:
  sample_transforms:
    - Decode: {}
    - RandomDistort: {}
    - RandomFlip: {prob: 0.5}
    - Mosaic: {prob: 0.5}
    # - RandomCrop: {}  <-- [삭제] 이미 640 전처리가 되어있으므로 불필요
    - Resize: {target_size: *eval_size, keep_ratio: False, interp: 2}
    - NormalizeImage: {is_scale: true, mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]}
    - Permute: {}

  batch_transforms:
    - PadBatch: {pad_to_stride: 32}
    - PadGT: {}
  batch_size: 32
  shuffle: true
  drop_last: true
  use_shared_memory: false

EvalReader:
  sample_transforms:
    - Decode: {}
    - Resize: {target_size: *eval_size, keep_ratio: False, interp: 2}
    - NormalizeImage: {is_scale: true, mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]}
    - Permute: {}
  batch_size: 1

TestReader:
  sample_transforms:
    - Decode: {}
    - Resize: {target_size: *eval_size, keep_ratio: False, interp: 2}
    - NormalizeImage: {is_scale: true, mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]}
    - Permute: {}
  batch_size: 1

LearningRate:
  base_lr: 0.02
  schedulers:
    - !CosineDecay
      max_epochs: 100
    - !LinearWarmup
      start_factor: 0.1
      steps: 300

OptimizerBuilder:
  optimizer:
    momentum: 0.9
    type: Momentum
  regularizer:
    factor: 0.00004
    type: L2
"""

with open(config_path, 'w') as f:
    f.write(config_content)

print(f"✅ 최신 버전 호환 설정 파일 생성 완료: {config_path}")

✅ 최신 버전 호환 설정 파일 생성 완료: /content/PP-PicoDet_dataset/custom_picodet.yml


In [ ]:
!python tools/train.py \
    -c /content/PP-PicoDet_dataset/custom_picodet.yml \
    --eval \
    -o save_dir="/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model_m_416_증강후"

[02/03 03:37:38] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
[02/03 03:37:38] ppdet.data.source.coco INFO: Load [4632 samples valid, 1 samples invalid] in file /content/PP-PicoDet_dataset/annotations/instances_train.json.
W0203 03:37:38.438906 122734 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 12.4, Runtime API Version: 11.7
W0203 03:37:38.439973 122734 gpu_resources.cc:164] device: 0, cuDNN Version: 9.2.
[02/03 03:37:39] ppdet.utils.checkpoint INFO: Finish loading model weights: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model_m_416/best_model/best_416_m_before.pdparams
[02/03 03:37:43] ppdet.engine.callbacks INFO: Epoch: [0] [  0/144] learning_rate: 0.002000 loss_vfl: 0.530221 loss_bbox: 0.438024 loss_dfl: 0.160433 loss: 1.128678 eta: 14:53:56 batch_cost: 3.7247 d

In [ ]:
checkpoint = "/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model_m_416/4"

print(f"🔄 학습을 이어갑니다: {checkpoint}")

# 3. 학습 재개
!python tools/train.py \
    -c /content/PP-PicoDet_dataset/custom_picodet.yml \
    --eval \
    -r "$checkpoint" \
    -o save_dir="/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model_m_416"

🔄 학습을 이어갑니다: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model_m_416/4
[02/03 00:15:17] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
[02/03 00:15:17] ppdet.data.source.coco INFO: Load [4632 samples valid, 1 samples invalid] in file /content/PP-PicoDet_dataset/annotations/instances_train.json.
W0203 00:15:17.314433 14397 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 12.4, Runtime API Version: 11.7
W0203 00:15:17.315485 14397 gpu_resources.cc:164] device: 0, cuDNN Version: 9.2.
[02/03 00:15:18] ppdet.utils.checkpoint INFO: Finish resuming model weights: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model_m_416/4.pdparams
[02/03 00:15:25] ppdet.engine.callbacks INFO: Epoch: [5] [ 0/72] learning_rate: 0.079985 loss_vfl: 0.830548 loss_bbox: 0.745546 loss_dfl: 0.285828 loss: 1.86

In [ ]:
import glob
import os

path = '/content/PP-PicoDet_dataset/images_640/train'
jpg_files = glob.glob(os.path.join(path, '*.jpg'))

print(f"JPG 파일 개수: {len(jpg_files)}개")

JPG 파일 개수: 4633개


In [ ]:
!python tools/export_model.py \
  -c /content/PP-PicoDet_dataset/custom_picodet.yml \
  -o weights=output/best_model.pdparams \
  --output_dir=inference_model

[02/02 13:25:41] ppdet.utils.checkpoint INFO: Skipping import of the encryption module.
W0202 13:25:41.334017 67551 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 12.4, Runtime API Version: 11.7
W0202 13:25:41.334962 67551 gpu_resources.cc:164] device: 0, cuDNN Version: 9.2.
[02/02 13:25:42] ppdet.utils.checkpoint INFO: Finish loading model weights: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/best_model.pdparams
[02/02 13:25:42] train INFO: Found 386 inference images in total.
[02/02 13:25:42] ppdet.data.source.category WARNING: anno_file 'None' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[02/02 13:25:42] ppdet.data.source.category WARNING: metric_type: COCO, load default categories of COCO.
100% 386/386 [00:11<00:00, 33.86it/s]
[02/02 13:25:54] ppdet.utils.download INFO: Downloading simfang.ttf from h

In [ ]:
import shutil

In [ ]:
src_dir = "/content/PaddleDetection/output/infer"

# 대상 Drive 폴더
dst_dir = "/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/그래빗(GrabIT)/PP-PicoDet/output"

# 1️⃣ 대상 폴더 없으면 생성
os.makedirs(dst_dir, exist_ok=True)

# 2️⃣ infer 폴더 통째로 복사
shutil.copytree(
    src_dir,
    os.path.join(dst_dir, "infer"),
    dirs_exist_ok=True
)

print("✅ infer 폴더 Drive로 복사 완료")

✅ infer 폴더 Drive로 복사 완료
